# ECE-GY 9163 Lab1 Guandong Kou (gk1675)

The description of this lab is re-organized in this [Google doc](https://docs.google.com/document/d/1wp9a-ns_xwh2_x5FIJFrzo0JQZdOY9Dwtg9LXW8pYVw)

rename 'part10' to 'validation_set' at `lemm_stop` directory
```!mv part10 validation_set```

In [1]:
from glob import glob
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction import text
from string import punctuation
from collections import Counter
from math import log2
import re
import numpy as np
from heapq import nlargest
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.metrics import confusion_matrix

In [93]:
# load training and test dataset
data_path = 'data/lemm_stop/part*/'
spam_files = glob(data_path + 'spms*[0-9]*.txt')
ham_files = glob(data_path + '*[0-9]*msg[0-9]*.txt')
testset_path = 'data/lemm_stop/testset/'
ts_spam_files = glob(testset_path + 'spms*[0-9]*.txt')
ts_ham_files = glob(testset_path + '*[0-9]*msg[0-9]*.txt')

In [3]:
n_spam, n_ham = len(spam_files), len(ham_files)
n_total = n_spam + n_ham
p_spam, p_ham = n_spam / n_total, n_ham / n_total

n_ts_ham, n_ts_spam = len(ts_ham_files), len(ts_spam_files)
n_test = n_ts_ham + n_ts_spam

In [4]:
def get_word_counter(files):
    # get word counter from files
    remove_set = set(punctuation) | set(text.ENGLISH_STOP_WORDS)
    w_counter = Counter()
    for file in files:
        word_set = set()
        with open(file) as f:
            for line in f.readlines():
                for word in word_tokenize(line):
                    if word not in remove_set and re.match('[a-zA-Z]{2,}', word):
                        word_set.add(word.lower())
        for word in word_set:
            w_counter[word] += 1
    return w_counter

In [5]:
ham_word_counter = get_word_counter(ham_files)
spam_word_counter = get_word_counter(spam_files)
total_word_counter = ham_word_counter + spam_word_counter

In [45]:
def get_entropy(p):
    return -(p * log2(p) + (1-p) * log2(1-p))

In [46]:
def get_p_word(word):
    return total_word_counter[word] / n_total

In [47]:
def get_H_C_X(word):
    p_word = total_word_counter[word] / n_total
    p_ham_cond_word = (ham_word_counter[word] + 1) / (total_word_counter[word] + 2) # Laplacian smoothing is applied here
    p_ham_cond_not_word = (n_ham - ham_word_counter[word] + 1) / (n_total - total_word_counter[word] + 2) #
    return p_word * get_entropy(p_ham_cond_word) + (1-p_word) * get_entropy(p_ham_cond_not_word)

In [48]:
H_C = get_entropy(p_spam)
def get_IG(word):
    H_C_X = get_H_C_X(word)
    return H_C - H_C_X

In [60]:
# top 10 words with largest information gain 
n_largest_IG = nlargest(10, [(get_IG(w), w) for w in total_word_counter])
n_largest_IG

[(0.2014815280269308, 'language'),
 (0.1670505008347254, 'remove'),
 (0.16449082041657143, 'free'),
 (0.14515234460220117, 'linguistic'),
 (0.14253899233783307, 'university'),
 (0.11749366725618493, 'money'),
 (0.09944312815537659, 'click'),
 (0.09135660326353257, 'market'),
 (0.08584499224521047, 'business'),
 (0.0804619418318766, 'today')]

## Classification
- Bernoulli NB classifier with binary features
- Multinomial NB with binary features
- Multinomial NB with term frequency (TF) features

In [131]:
# n_features = 1000
n_features_max = 1000
top_n_features_map = {
    word: idx for idx, (_, word) in enumerate(
        nlargest(n_features_max, [(get_IG(w), w) for w in total_word_counter]))}

In [137]:
def count_words_to_matrix(files, M):
    for idx, file in enumerate(files):
        with open(file) as f:
            for line in f.readlines():
                for word in word_tokenize(line):
                    if word in top_n_features_map:
                        M[idx][top_n_features_map[word]] += 1

In [147]:
ytr = np.array([0] * n_ham + [1] * n_spam)
Xtr = np.zeros((n_total, n_features_max))
count_words_to_matrix(ham_files + spam_files, Xtr)

In [138]:
yts = np.array([0] * n_ts_ham + [1] * n_ts_spam)
Xts = np.zeros((n_test, n_features_max))
count_words_to_matrix(ts_ham_files + ts_spam_files, Xts)

In [148]:
def get_X_by_n_feat(X, n):
    return X[:, :n]

In [149]:
# Bernoulli NB classifier with binary features;
model_Bernoulli_NB = BernoulliNB()
for n_features in [10, 100, 1000]:
    _Xtr = get_X_by_n_feat(Xtr, n_features)
    Xtr_binary = np.array([[1 if v > 0 else 0 for v in row] for row in _Xtr])
    model_Bernoulli_NB.fit(Xtr_binary, ytr)
    _Xts = get_X_by_n_feat(Xts, n_features)
    Xts_binary = np.array([[1 if v > 0 else 0 for v in row] for row in _Xts])
    yts_hat = model_Bernoulli_NB.predict(Xts_binary)
    accuracy = np.mean(yts_hat == yts)
    print(accuracy, n_features)

0.9450171821305842 10
0.9484536082474226 100
0.9484536082474226 1000


In [152]:
# MulPnomial NB with binary features
model_Multinomial_NB = MultinomialNB()
for n_features in [10, 100, 1000]:
    Xtr_binary = np.array([[1 if v > 0 else 0 for v in row] for row in get_X_by_n_feat(Xtr, n_features)])
    model_Multinomial_NB.fit(Xtr_binary, ytr)
    Xts_binary = np.array([[1 if v > 0 else 0 for v in row] for row in get_X_by_n_feat(Xts, n_features)])
    yts_hat = model_Multinomial_NB.predict(Xts_binary)
    accuracy = np.mean(yts_hat == yts)
    print(accuracy, n_features)

0.9518900343642611 10
0.9828178694158075 100
0.9896907216494846 1000


In [154]:
# MulPnomial NB with term frequency (TF) features
model_Multinomial_NB = MultinomialNB()
for n_features in [10, 100, 1000]:
    model_Multinomial_NB.fit(Xtr, ytr)
    yts_hat = model_Multinomial_NB.predict(Xts)
    accuracy = np.mean(yts_hat == yts)
    print(accuracy, n_features)

0.9896907216494846 10
0.9896907216494846 100
0.9896907216494846 1000
